# Predicting vaccination rates in the US


## Models used:

- `Regularised regression`
- `Random Forest regression`
- `Gradient Boosted Decision Trees`
- `SVM`?
- `NN`?


### Missing values imputation?

- first order - mean/median
- similar to - value from grouping of other similar observations
- modelling?


### Normalisation:

- `Regularised regression`

